# Quick Start


## Context

Context is to help you manage multi round dialogues. For each round, you only need to append new message to it. It will auto track previous conversation history in 
context.state.messages

In [ ]:
%env ARK_API_KEY=<YOUR API KEY>

In [3]:
from arkitect.core.component.context.context import Context


MODEL = "deepseek-v3-241226"


async def context_chat():
    first_round_message = (
        "请记住你是一个ai助手，我是一个会飞的西兰花。请问巴黎是哪里的首都"
    )
    second_round_question = "请回答我是谁"
    third_round_question = "你在第一轮回答的是什么问题，回复是什么？"
    ctx = Context(model=MODEL)
    await ctx.init()

    print("\n" + "-" * 20 + "Start of first round" + "-" * 20 + "\n")
    completion = await ctx.completions.create(
        [{"role": "user", "content": first_round_message}], stream=True
    )
    async for chunk in completion:
        if chunk.choices:
            print(chunk.choices[0].delta.content, end="")
    print("\n" + "=" * 20 + "End of first round" + "=" * 20 + "\n")

    print("\n" + "-" * 20 + "Start of second round" + "-" * 20 + "\n")
    completion = await ctx.completions.create(
        [{"role": "user", "content": second_round_question}], stream=True
    )
    async for chunk in completion:
        if chunk.choices:
            print(chunk.choices[0].delta.content, end="")
    print("\n" + "=" * 20 + "End of second round" + "=" * 20 + "\n")

    print("\n" + "-" * 20 + "Start of third round" + "-" * 20 + "\n")
    completion = await ctx.completions.create(
        [{"role": "user", "content": third_round_question}], stream=True
    )
    async for chunk in completion:
        if chunk.choices:
            print(chunk.choices[0].delta.content, end="")
    print("\n" + "=" * 20 + "End of third round" + "=" * 20 + "\n")

In [4]:
await context_chat()

2025-04-10 17:51:38 [debug    ] singleton class initialized    name=ClientPool

--------------------Start of first round--------------------

当然，会飞的西兰花！巴黎是法国的首都。法国位于欧洲西部，是一个以其丰富的历史、文化、艺术和美食而闻名的国家。巴黎更是被誉为“光之城”，拥有许多世界著名的地标，如埃菲尔铁塔、卢浮宫和圣母院等。希望这些信息对你有所帮助！如果你有其他问题，随时告诉我。
====================End of first round====================


--------------------Start of second round--------------------

你是会飞的西兰花！一个独特而有趣的存在，拥有飞行的能力。有什么我可以帮助你的吗？
====================End of second round====================


--------------------Start of third round--------------------

在第一轮，我回答的问题是：“巴黎是哪里的首都”，我的回复是：

“当然，会飞的西兰花！巴黎是法国的首都。法国位于欧洲西部，是一个以其丰富的历史、文化、艺术和美食而闻名的国家。巴黎更是被誉为‘光之城’，拥有许多世界著名的地标，如埃菲尔铁塔、卢浮宫和圣母院等。希望这些信息对你有所帮助！如果你有其他问题，随时告诉我。”

你还有其他问题吗？😊
====================End of third round====================



注意到这里每次只需要append新的user message，之前的user message 和llm 回复都自动append在context里 （准确的说是context.state.messages里）

## Context With tool

你可以在context中传入一个tools参数，Context在运行中，如果有遇到toolcall 就会直接执行tool然后将结果append在后面并继续运行。Tool 可以是任何一个你已经写好的python方法

In [7]:
from arkitect.core.component.context.context import Context
from arkitect.core.component.context.model import ToolChunk

# 在这里创建你的工具方法
# 任何的python Callable （sync or async）都可以 直接作为tool使用


def get_weather(city: str, next_n_days: int) -> str:
    """get the weather of a city

    Args:
        city (str): city name
        next_n_days (int): next n days. Need to be a positive integer.

    Returns:
        str: description of next_n_days' weather
    """
    return "Weather at {} is sunny".format(city)


async def context_chat_with_tools():
    first_round_message = "请帮我查一下北京未来两天的天气"
    ctx = Context(
        model=MODEL,
        tools=[
            get_weather
        ],  # 直接在这个list里传入你的所有python 方法作为tool，tool的描述会自动带给模型推理，tool的执行在ctx.completions.create 中会自动进行
    )
    await ctx.init()

    completion = await ctx.completions.create(
        [{"role": "user", "content": first_round_message}], stream=True
    )
    async for chunk in completion:
        if isinstance(chunk, ToolChunk):
            continue
        else:
            print(chunk.choices[0].delta.content, end="")

In [8]:
await context_chat_with_tools()

[04/10/25 17:53:55] INFO     HTTP Request: POST                                                     ]8;id=287128;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=385183;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               

[04/10/25 17:53:56] INFO     HTTP Request: POST                                                     ]8;id=769991;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=559538;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               

北京未来两天的天气情况如下：

- **第一天**: 晴天

请注意，天气情况可能会有所变化，建议您随时关注最新的天气预报。如果您需要更详细的信息，请告诉我！

观察到这里有两次LLM的调用 （两次https://ark.cn-beijing.volces.com/api/v3/chat/completions的请求日志），说明模型第一次推理完之后，返回了带tool_call 的字段，自动执行了get_weather 工具之后，自动把get_weather的结果填充到了messages中，第二次调用LLM的时候，就依照tool的返回结果，自动填充到了messages中。

## Hooks in context

由于context中封装了过多步骤，或许连续的LLM 调用和tool call调用并不能满足debug的需求，因此在这个自动调用的过程中，有几个关键的生命周期hook，可以帮助实现打印日志，或者在调用LLM 前做一些预处理，或者human in the loop的需求

首先要理解 在context中处理的流程大概如下

1. 如果context中末尾的消息带有tool_call，则会先进行 tool call调用，并把结果append在context中
2. for loop
   2.1 执行LLM 调用
   2.2. 把 结果append在context中
   2.3. if 有tool-call
      2.3.1. 执行tool-call
      2.3.2. 把结果append在context中
      2.3.3. goto 1
   2.4. else: 结束

在这个过程中，有4个关键的生命周期节点
- pre llm call
- post llm call
- pre tool call
- post tool call

你可以在这几个节点中做一些预处理，或者打印日志，或者human in the loop的需求


In [11]:
from typing import Any, Optional
from arkitect.core.component.context.context import Context
from arkitect.core.component.context.model import State

from arkitect.core.component.context.hooks import (
    PreToolCallHook,
    PostToolCallHook,
    PreLLMCallHook,
    PostLLMCallHook,
)


class MyHooks(PreToolCallHook, PostToolCallHook, PreLLMCallHook, PostLLMCallHook):
    async def pre_tool_call(
        self,
        name: str,
        arguments: str,
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside pre tool call" + "=" * 20 + "\n")
        last_assistant_message = state.messages[-1]
        tool_call_part = last_assistant_message["tool_calls"]
        for tool_call in tool_call_part:
            print(
                f"Tool {tool_call['function']['name']} with {tool_call['function']['arguments']}"
            )
            # you may modify this or ask users for approval here
        return state  # return state no matter if have modified it

    async def post_tool_call(
        self,
        name: str,
        arguments: str,
        response: Any,
        exception: Optional[Exception],
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside post tool call" + "=" * 20 + "\n")
        print(f"Tool {name} with {arguments} returned {response}")
        return state  # return state no matter if have modified it

    async def pre_llm_call(
        self,
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside pre llm call" + "=" * 20 + "\n")
        print(f"Sending into LLM: {state.messages}")
        return state  # return state no matter if have modified it

    async def post_llm_call(
        self,
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside post llm call" + "=" * 20 + "\n")
        print(f"Output from LLM: {state.messages}")
        return state  # return state no matter if have modified it


# Let's continue to use the previous example.
# But this time we will add in the four hooks
async def context_chat_with_tools_with_hooks():
    first_round_message = "请帮我查一下北京未来两天的天气"
    ctx = Context(
        model=MODEL,
        tools=[
            get_weather
        ],  # 直接在这个list里传入你的所有python 方法作为tool，tool的描述会自动带给模型推理，tool的执行在ctx.completions.create 中会自动进行
    )
    my_hook = MyHooks()
    ctx.set_post_llm_call_hook(my_hook)
    ctx.set_pre_tool_call_hook(my_hook)
    ctx.set_post_tool_call_hook(my_hook)
    ctx.set_pre_llm_call_hook(my_hook)
    await ctx.init()

    completion = await ctx.completions.create(
        [{"role": "user", "content": first_round_message}], stream=True
    )
    async for chunk in completion:
        if isinstance(chunk, ToolChunk):
            continue
        else:
            print(chunk.choices[0].delta.content, end="")

In [12]:
await context_chat_with_tools_with_hooks()


====================Inside pre llm call====================

Sending into LLM: [{'role': 'user', 'content': '请帮我查一下北京未来两天的天气'}]


[04/10/25 17:54:43] INFO     HTTP Request: POST                                                     ]8;id=655844;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=158570;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               


====================Inside post llm call====================

Output from LLM: [{'role': 'user', 'content': '请帮我查一下北京未来两天的天气'}, {'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'index': 0, 'id': 'call_c07rp120axnb77s4bws95krt', 'function': {'arguments': '{"city":"北京","next_n_days":2}', 'name': 'get_weather'}, 'type': 'function'}], 'audio': None, 'reasoning_content': None}]

====================Inside pre tool call====================

Tool get_weather with {"city":"北京","next_n_days":2}

====================Inside post tool call====================

Tool get_weather with {"city":"北京","next_n_days":2} returned Weather at 北京 is sunny

====================Inside pre llm call====================

Sending into LLM: [{'role': 'user', 'content': '请帮我查一下北京未来两天的天气'}, {'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'index': 0, 'id': 'call_c07rp120axnb77s4bws95krt', 'function': {'arguments': '{"city":"北京","next_n_days":2}', 'name': 'get_weath

[04/10/25 17:54:44] INFO     HTTP Request: POST                                                     ]8;id=148049;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=99190;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               

北京未来两天的天气如下：

- 第一天：晴
- 第二天：晴

这两天北京都是晴朗的天气，适合外出活动。
====================Inside post llm call====================

Output from LLM: [{'role': 'user', 'content': '请帮我查一下北京未来两天的天气'}, {'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'index': 0, 'id': 'call_c07rp120axnb77s4bws95krt', 'function': {'arguments': '{"city":"北京","next_n_days":2}', 'name': 'get_weather'}, 'type': 'function'}], 'audio': None, 'reasoning_content': None}, {'role': 'tool', 'tool_call_id': 'call_c07rp120axnb77s4bws95krt', 'content': 'Weather at 北京 is sunny'}, {'content': '北京未来两天的天气如下：\n\n- 第一天：晴\n- 第二天：晴\n\n这两天北京都是晴朗的天气，适合外出活动。', 'role': 'assistant', 'function_call': None, 'tool_calls': [], 'audio': None, 'reasoning_content': None}]


## Working with MCP servers

在tools 参数中，除了python functions，你也可以直接传入SDK中的 MCPClient。
创建MCPClient的方式有两种这里都会简单demo

首先启动你的MCP server 

    ```bash
    export ARK_API_KEY=<YOUR API KEY> 
    export ARK_TOOL_CACULATOR=true 
    export PORT=8765 
    uvx --from git+https://github.com/volcengine/ai-app-lab.git#subdirectory=mcp/server/mcp_server_ark mcp-server-ark --transport sse
    ```

这里我们以一个计算器MCP server 为例

1. 直接创建MCPClient（不推荐）：如果要直接创建一个MCPClient以SSE的方式连接到一个服务，可以参考下面的写法

In [17]:
from typing import Any, Optional
from arkitect.core.component.context.context import Context
from arkitect.core.component.context.model import State
from arkitect.core.component.tool.mcp_client import MCPClient

from arkitect.core.component.context.hooks import (
    PreToolCallHook,
    PostToolCallHook,
)


class MyHooks(PreToolCallHook, PostToolCallHook):
    async def pre_tool_call(
        self,
        name: str,
        arguments: str,
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside pre tool call" + "=" * 20 + "\n")
        last_assistant_message = state.messages[-1]
        tool_call_part = last_assistant_message["tool_calls"]
        for tool_call in tool_call_part:
            print(
                f"Tool {tool_call['function']['name']} with {tool_call['function']['arguments']}"
            )
            # you may modify this or ask users for approval here
        return state  # return state no matter if have modified it

    async def post_tool_call(
        self,
        name: str,
        arguments: str,
        response: Any,
        exception: Optional[Exception],
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside post tool call" + "=" * 20 + "\n")
        print(f"Tool {name} with {arguments} returned {response}")
        return state  # return state no matter if have modified it


#
async def context_chat_with_tools_with_mcp_clients():
    mcp_client = MCPClient(name="Calculator", server_url="http://localhost:8765/sse")
    first_round_message = "请你计算一下12341234 /7823847+999"
    ctx = Context(
        model=MODEL,
        tools=[
            mcp_client
        ],  # 直接在这个list里传入你的所有的python方法或者MCPClient，可以混着传入
    )
    my_hook = MyHooks()
    ctx.set_pre_tool_call_hook(my_hook)
    ctx.set_post_tool_call_hook(my_hook)
    await ctx.init()

    completion = await ctx.completions.create(
        [{"role": "user", "content": first_round_message}], stream=True
    )
    async for chunk in completion:
        if isinstance(chunk, ToolChunk):
            continue
        else:
            print(chunk.choices[0].delta.content, end="")
    await mcp_client.cleanup()  # 注意cleanup！！！

In [18]:
await context_chat_with_tools_with_mcp_clients()

[04/10/25 18:02:00] WARNING  MCP client is not connected to server yet. Connecting...             ]8;id=216704;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/mcp_client.py\mcp_client.py]8;;\:]8;id=23173;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/mcp_client.py#184\184]8;;\

                    INFO     Connecting to SSE endpoint: http://localhost:8765/sse                        ]8;id=71372;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=183099;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8765/sse "HTTP/1.1 200 OK"          ]8;id=995821;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=857690;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received endpoint URL:                                                       ]8;id=428233;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=369154;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py#65\65]8;;\
                             http://localhost:8765/messages/?session_id=0b87703af22843bc94801d985f10b9fc           

                    INFO     Starting post writer with endpoint URL:                                     ]8;id=711872;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=541974;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py#135\135]8;;\
                             http://localhost:8765/messages/?session_id=0b87703af22843bc94801d985f10b9fc           

...


====================Inside pre tool call====================

Tool caculator with {"input":"12341234 /7823847+999"}


                    INFO     HTTP Request: POST                                                     ]8;id=776328;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=74706;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8765/messages/?session_id=0b87703af22843bc94801d985f1                
                             0b9fc "HTTP/1.1 202 Accepted"                                                         


====================Inside post tool call====================

Tool caculator with {"input":"12341234 /7823847+999"} returned {"msg": "Good", "result": "7828364387 / 7823847"}


[04/10/25 18:02:03] INFO     HTTP Request: POST                                                     ]8;id=93813;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=76662;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               

通过计算，12341234 / 7823847 + 999 的结果是：

\[
\frac{7828364387}{7823847}
\]

如果需要进一步的简化或转换为小数形式，请告诉我！

2. 通过配置文件创建：你也可以把你的MCPClient以配置化的形式放在一个JSON文件中，sdk中有方法可以帮你一件拉起（更推荐的做法）

In [21]:
from typing import Any, Optional
from arkitect.core.component.context.context import Context
from arkitect.core.component.context.model import State
from arkitect.core.component.tool.builder import build_mcp_clients_from_config

from arkitect.core.component.context.hooks import (
    PreToolCallHook,
    PostToolCallHook,
)


class MyHooks(PreToolCallHook, PostToolCallHook):
    async def pre_tool_call(
        self,
        name: str,
        arguments: str,
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside pre tool call" + "=" * 20 + "\n")
        last_assistant_message = state.messages[-1]
        tool_call_part = last_assistant_message["tool_calls"]
        for tool_call in tool_call_part:
            print(
                f"Tool {tool_call['function']['name']} with {tool_call['function']['arguments']}"
            )
            # you may modify this or ask users for approval here
        return state  # return state no matter if have modified it

    async def post_tool_call(
        self,
        name: str,
        arguments: str,
        response: Any,
        exception: Optional[Exception],
        state: State,
    ) -> State:
        print("\n" + "=" * 20 + "Inside post tool call" + "=" * 20 + "\n")
        print(f"Tool {name} with {arguments} returned {response}")
        return state  # return state no matter if have modified it


CONFIG_FILE_PATH = "./mcp_config.json"


async def context_chat_with_tools_with_mcp_config_file():
    mcp_clients, cleanup = build_mcp_clients_from_config(CONFIG_FILE_PATH)
    first_round_message = "请你计算一下12341234 /7823847+999"
    ctx = Context(
        model=MODEL,
        tools=list(
            mcp_clients.values()
        ),  # 直接在这个list里传入你的所有的python方法或者MCPClient，可以混着传入
    )
    my_hook = MyHooks()
    ctx.set_pre_tool_call_hook(my_hook)
    ctx.set_post_tool_call_hook(my_hook)
    await ctx.init()

    completion = await ctx.completions.create(
        [{"role": "user", "content": first_round_message}], stream=True
    )
    async for chunk in completion:
        if isinstance(chunk, ToolChunk):
            continue
        else:
            print(chunk.choices[0].delta.content, end="")
    await cleanup()  # 注意cleanup！！！

In [23]:
await context_chat_with_tools_with_mcp_config_file()

[04/10/25 18:03:11] INFO     Starting server                                                          ]8;id=109070;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/builder.py\builder.py]8;;\:]8;id=865607;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/builder.py#59\59]8;;\

                    WARNING  MCP client is not connected to server yet. Connecting...             ]8;id=32664;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/mcp_client.py\mcp_client.py]8;;\:]8;id=977722;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/arkitect/core/component/tool/mcp_client.py#184\184]8;;\

                    INFO     Connecting to SSE endpoint: http://localhost:8765/sse                        ]8;id=944757;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=317390;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py#45\45]8;;\

                    INFO     HTTP Request: GET http://localhost:8765/sse "HTTP/1.1 200 OK"          ]8;id=846185;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=762803;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received endpoint URL:                                                       ]8;id=569242;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py\sse.py]8;;\:]8;id=220136;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/client/sse.py#65\65]8;;\
                             http://localhost:8765/messages/?session_id=9bc23bc008f74fc28a6d3df8073f3ead           

...

                    INFO     HTTP Request: POST                                                     ]8;id=554263;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=863944;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             http://localhost:8765/messages/?session_id=9bc23bc008f74fc28a6d3df8073                
                             f3ead "HTTP/1.1 202 Accepted"                                                         


====================Inside post tool call====================

Tool caculator with {"input":"7828364387 / 7823847"} returned {"msg": "Good", "result": "7828364387 / 7823847"}


[04/10/25 18:03:18] INFO     HTTP Request: POST                                                     ]8;id=470638;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=418165;file:///Users/bytedance/miniconda3/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://ark.cn-beijing.volces.com/api/v3/chat/completions "HTTP/1.1                   
                             200 OK"                                                                               

其结果是一个分数，我们可以将其转换为小数形式以便更直观地理解。

7828364387 / 7823847 ≈ 1000.999

因此，最终的计算结果约为 **1000.999**。

[04/10/25 18:03:19] ERROR    Task exception was never retrieved                                 ]8;id=594739;file:///Users/bytedance/miniconda3/lib/python3.12/asyncio/base_events.py\base_events.py]8;;\:]8;id=770617;file:///Users/bytedance/miniconda3/lib/python3.12/asyncio/base_events.py#1819\1819]8;;\
                             future: <Task finished name='Task-67'                                                 
                             coro=<<async_generator_athrow without __name__>()>                                    
                             exception=RuntimeError('Attempted to exit cancel scope in a                           
                             different task than it was entered in')>                                              
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/c │                    
                             │ lient/sse.py:141 in sse_client                                 │                    
                             │                                                                │                    
                             │   138 │   │   │   │   │   tg.start_soon(post_writer, endpoint_ │                    
                             │   139 │   │   │   │   │                                        │                    
                             │   140 │   │   │   │   │   try:                                 │                    
                             │ ❱ 141 │   │   │   │   │   │   yield read_stream, write_stream  │                    
                             │   142 │   │   │   │   │   finally:                             │                    
                             │   143 │   │   │   │   │   │   tg.cancel_scope.cancel()         │                    
                             │   144 │   │   finally:                                         │                    
                             ╰────────────────────────────────────────────────────────────────╯                    
                             GeneratorExit                                                                         
                                                                                                                   
                             During handling of the above exception, another exception                             
                             occurred:                                                                             
                                                                                                                   
                             ╭────────────── Traceback (most recent call last) ───────────────╮                    
                             │ /Users/bytedance/miniconda3/lib/python3.12/site-packages/mcp/c │                    
                             │ lient/sse.py:43 in sse_client                                  │                    
                             │                                                                │                    
                             │    40 │   read_stream_writer, read_stream = anyio.create_memor │                    
                             │    41 │   write_stream, write_stream_reader = anyio.create_mem │                    
                             │    42 │                                                        │                    
                             │ ❱  43 │   async with anyio.create_task_group() as tg:          │                    
                             │    44 │   │   try:                                             │                    
                             │    45 │   │   │   logger.info(f"Connecting to SSE endpoint: {r │                    
                             │    46 │   │   │   async with httpx.AsyncClient(headers=headers